In [ ]:
%matplotlib inline

In [ ]:
from datetime import datetime
import pathlib
import time
import json
from libcomcat.search import search, get_event_by_id
import pandas as pd
from IPython.display import display
from urllib.request import urlopen
import matplotlib.pyplot as plt

# Pandas

In this notebook we will explore the pandas Python library, and how it can speed up data analysis.

# Table of contents
1. [Defining Data](#defining-data)
2. [Loading Data](#loading-data)
3. [Examining Data](#examining-data)
4. [Selecting Data](#selecting-data)
    1. [Series Object](#series-object)
    2. [Series Data](#series-data)
    3. [Index Definition](#index-definition)
    4. [Selecting Data by Column](#selecting-data-by-column)
    5. [Selecting Data by Row](#selecting-data-by-row)
5. [Create Significant Events](#create-significant-events)
    1. [Removing Missing Data](#removing-missing-data)
    2. [Select Rows by Multiple Criteria](#select-rows-by-multiple-criteria)
6. [Saving Results](#saving-results)
7. [Plotting Results](#plotting-results)
8. [Working with String Data](#working-with-string-data)
9. [Applying Functions to Data](#applying-functions-to-data)
10. [Resources](#resources)

## Defining Data  <a name="defining-data"></a>

Philosophically, the term "data" is heavily overloaded:

 - To a scientist, data can mean raw information collected by a human or instrument in the lab or field
 - To a programmer, who lives in a binary world, data are those bits of information that are NOT CODE.
 - To a GIS analyst, data is **geospatial data** - vectors and rasters.
 - To a business analyst, data can mean (sometimes sensitive) information collected about customers

When we talk about "data" in the context of this tutorial, we are talking about two-dimensional tabular information, like you would see in a spreadsheet or CSV file. These data could be data collected in the field, the results of a model, the results of a geospatial analysis, or information about customers that may be sensitive or be subject to privacy concerns.

## Loading Data <a name="loading-data"></a>

pandas provides a number of methods for getting data into our structure of primary interest, the *dataframe*.

 - Excel
 - CSV/tab separated/other delimiter
 - Relational Databases (MySQL, SQLite, etc)
 - Fixed width format data
 - Manual creation
 
 I've written a function here to grab product "properties" from ComCat, using it's API. The function saves these 
 results in a CSV. We'll load that CSV file into a dataframe.

In [ ]:
events_file = 'events_2020_properties.csv'

In [ ]:
def get_properties_dataframe(starttime, endtime, products, 
                             minmag=5.5, maxmag=9.9):
    '''
    Download product properties for input products.
    
    Args:
        starttime (datetime): Python datetime for search start time.
        endtime (datetime): Python datetime for search end time.
        products (list): List of strings of valid ComCat product types 
                        ('shakemap', 'losspager', 'dyfi', 'finite-fault')
        minmag (float): Minimum magnitude for search.
        maxmag (float): Maximum magnitude for search.
    Returns:
        dataframe: pandas Dataframe with fixed columns:
                   - eventid
                   - time
                   - location
                   - latitude
                   - longitude
                   - depth
                   - magnitude
                   
                   and then any number of variable columns of the different product properties.
    '''
    TIMEFMT = '%Y-%m-%dT%H:%M:%S'
    starttime_str = starttime.strftime(TIMEFMT)
    endtime_str = endtime.strftime(TIMEFMT)
    url = ('https://earthquake.usgs.gov/fdsnws/event/1/query?'
           f'format=geojson&starttime={starttime_str}&'
           f'endtime={endtime_str}&orderby=time-asc&'
           f'minmagnitude={minmag}&maxmagnitude={maxmag}')
    with urlopen(url) as fh:
        data = fh.read().decode('utf8')
    jdict = json.loads(data)
    rows = []
    for feature in jdict['features']:
        row = {}    
        types = feature['properties']['types'].split(',')[1:-1]
        ptypes = set(products) & set(types) # types we want intersected w/ what we have
        if not len(ptypes):
            mag = feature['properties']['mag']
            continue
        detail_url = feature['properties']['detail']
        with urlopen(detail_url) as fh:
            data = fh.read().decode('utf8')
        detail = json.loads(data)
        row['eventid'] = detail['id']
        
        # get event time from Unix timestamp in milliseconds
        dtime = detail['properties']['time']
        row['time'] = datetime.utcfromtimestamp(dtime/1000)
        
        row['location'] = detail['properties']['place']
        row['latitude'] = detail['geometry']['coordinates'][1]
        row['longitude'] = detail['geometry']['coordinates'][0]
        row['depth'] = detail['geometry']['coordinates'][2]
        row['magnitude'] = detail['properties']['mag']
        
        for ptype in ptypes:
            product = detail['properties']['products'][ptype][0]
            for pkey in product['properties']:
                key = f'{ptype}-{pkey}'
                value = product['properties'][pkey]
                try:
                    value = int(value)
                except:
                    try:
                        value = float(value)
                    except:
                        pass
                if value == 'None':
                    value = float('nan')
                row[key] = value
        rows.append(row)
    dataframe = pd.DataFrame(rows)
    return dataframe

Let's retrieve a local file, if it exists. We'll show the first five rows of the dataframe when we're done. Note that there are a *lot* of columns, probably more than we need.

In [ ]:
if not pathlib.Path(events_file).exists():
    t1=time.time()
    products = ['shakemap', 'losspager', 'dyfi']
    starttime = datetime(2020, 1, 1)
    endtime = datetime(2021, 1, 1)
    props = get_properties_dataframe(starttime, endtime, products, minmag=5.5)
    t2 = time.time()
    dt2 = t2-t1
    print(f'{dt2:.1f} seconds to retrieve dataframe')
    pd.set_option('display.max_columns', None)
    # save the props dataframe to csv
    props.to_csv(events_file, index=False)
else:
    props = pd.read_csv(events_file, sep=',', header=0, parse_dates=['time'])
props.head()

## Examining Data <a name="examining-data"></a>

In [ ]:
# Get the size of the dataframe
# props.shape
# len(props)

# look at the columns
# props.columns

# look at the indices
# props.index

# Look at the whole data set
# props

# Look at a subset of the data (by row)
# props.head(5)

## Selecting Data <a name="selecting-data"></a>

There are multiple ways to select data from a DataFrame in either dimension. Here we'll explain a few of these methods and explain some important attributes along the way.

###  Series Object <a name="series-object"></a>

A pandas DataFrame, as we've seen is made up of rows and columns of data, like a spreadsheet. You might be wondering what data structure is returned if you extract a single column or row. The answer is a **Series** object, which is interesting in it's own right for time series analysis, as just one example.

In [ ]:
magnitudes = props['magnitude']
magnitudes

### Series Data <a name="series-data"></a>

A Series object is a wrapper around a numpy array, and it implements many of the same basic functions as a numpy array. This numpy array is accessible through the Series `values` property.

In [ ]:
print(f'Magnitudes sum: {magnitudes.sum():.2f}')
print(f'Magnitudes mean: {magnitudes.mean():.2f}')
print(f'Magnitudes std: {magnitudes.std():.2f}')
print(f'Magnitudes min: {magnitudes.min():.2f}')
print(f'Magnitudes max: {magnitudes.max():.2f}')
print(f'Magnitudes array: {magnitudes.values}')

### Index Definition <a name="index-definition"></a>

In the same way that a column name identifies a *column*, an index is a way to uniquely identify a *row*. 

Indices might be the following things:

 - In **medicine**, an index might be a patient ID
 - In **business**, an index might be an invoice ID, or a customer ID
 - In **seismology**, an index might be the ComCat ID, or a seismic station ID.
 
That being said, I tend not to use the index capability, because searching for rows based on specific criteria is typically a more useful approach - few people internalize or remember ComCat IDs, and you may very easily have more than one row with the same seismic station in your dataset.

### Selecting Data by Column <a name="selecting-data-by-column"></a>

You can select data from one or more dataframe columns, or one or more rows. Row selection can be done in a few different ways.

In [ ]:
# selecting by columns
keep_columns = ['eventid', 'time', 
                'latitude', 'longitude', 
                'depth', 'magnitude'
               ]
props[keep_columns]

### Selecting Data by Row <a name="selecting-data-by-row"></a>

You can select data by row in three different ways:

 - Select by index
 - Select by row location
 - Select by column criteria
 
 In order to demonstrate that the row **index** can be different than row **location**, I'll go out of order and create a subset of data first by selecting events with at least a magnitude of 7.5. Later we'll go into more detail on selecting data from rows.

In [ ]:
# select by simple criteria
mag_criteria = props['magnitude'] >= 7.5
subframe = props[mag_criteria]
subframe

Note that the rows in this data subset retained their index. Now let's select rows from this subset by that *index*.

In [ ]:
subframe.loc[[39,92]]

Select rows from this subset by row *location*

In [ ]:
subframe.iloc[0:2]

## Create Significant Events <a name="create-significant-events"></a>

Let's select down from all of these product properties some that are interesting for further analysis. Here we're selecting the following columns:

 - **shakemap-maxmmi** The maximum Mercalli intensity for an event as modeled by ShakeMap
 - **losspager-alertlevel** The PAGER alert level (Green, Yellow, Orange, Red) assigned to the event
 - **dyfi-num-responses** The number of DYFI responses that came in from users

In [ ]:
keep_columns = ['eventid', 'time', 
                'latitude', 'longitude', 
                'depth', 'magnitude', 
                'shakemap-maxmmi', 'losspager-alertlevel',
                'dyfi-num-responses'
               ]
sprops = props[keep_columns]
sprops.head()

### Removing Missing Data <a name="removing-missing-data"></a>

Notice above that some earthquakes do not have all columns set to real values - where this occurs, pandas inserts *NaN* values. pandas provides a method to allow us to remove rows with NaNs present in any of the columns. Let's remove all the rows where *any* of our properties of interest is set to NaN.

In [ ]:
# clean up events that do not have DYFI or PAGER results
print(f'Events before filter: {len(sprops)}')
columns = ['losspager-alertlevel', 'dyfi-num-responses', 'shakemap-maxmmi']
sprops = sprops.dropna(subset=columns)
print(f'Events after filter: {len(sprops)}')
sprops.head()

### Select Rows by Multiple Criteria <a name="select-rows-by-multiple-criteria"></a>

Selecting rows based on single or multiple column criteria will be a common goal. Imagine that we want to select down a list of "significant" events from 2020, based on four criteria:

 - **Magnitude** >= 7.0
 - **Maximum MMI** >= 7.0
 - **Number of DYFI Responses** >= 100
 - **PAGER Alert Level** >= Yellow
 
 We'll go through each of these in turn, and then see how to combine them to create our dataset of significant events.

In [ ]:
condition_magnitude = sprops['magnitude'] >= 6.0
sprops.loc[condition_magnitude].head()

What does this condition look like? It turns out it is a **Series** object with boolean values at each index. 

In [ ]:
condition_magnitude

In [ ]:
condition_maxmmi = sprops['shakemap-maxmmi'] >= 6.0
sprops.loc[condition_maxmmi].head()

In [ ]:
condition_numresp = sprops['dyfi-num-responses'] > 100
sprops.loc[condition_numresp].head()

In [ ]:
condition_yellow = sprops['losspager-alertlevel'].str.match('yellow')
condition_orange = sprops['losspager-alertlevel'].str.match('orange')
condition_red = sprops['losspager-alertlevel'].str.match('red')

# you can combine these conditions with "and" or "or" conditions
combined_condition = condition_yellow | condition_orange | condition_red
sprops.loc[combined_condition].head()

In [ ]:
condition_alert = sprops['losspager-alertlevel'].isin(['yellow', 'orange', 'red'])
sprops.loc[condition_alert].head()

Finally, we can combine all of these conditions together, creating a list of significant events from 2020.

In [ ]:
# combine them all together
# recall: M >= 6 OR maxmmi >= 6 OR NumResp >= 100 OR Alert > GREEN
combined_condition = condition_magnitude & condition_maxmmi & condition_numresp & condition_alert
significant = sprops.loc[combined_condition]
significant

In [ ]:
len(significant)

## Saving Results <a name="saving-results"></a>

In [ ]:
# index=False indicates that we do not want to save the index "column" to the output format. Feel free to keep 
# the index if it is useful in your work.
significant.to_csv('significant_events_2020.csv', index=False)
significant.to_excel('significant_events_2020.xlsx', index=False)

## Plotting Results <a name="plotting-results"></a>

pandas also incorporates matplotlib plotting capabilities within it, so it is possible to make a number of plots from your dataset without calling matplotlib functions directly.

In [ ]:
# sprops['losspager-alertlevel'].plot.bar()
ax = sprops['losspager-alertlevel'].value_counts(sort=False).plot.bar(title='Alert Level Counts', figsize=(15,7))
# ax.get_xlim()
ax.text(1.0, 175, f'N={len(sprops)}', fontsize=18)

In [ ]:
ax = sprops.plot(x='magnitude', y='shakemap-maxmmi', kind='scatter', figsize=(15,7.5), title = 'Mag vs MMI')

In [ ]:
ax = props['magnitude'].hist(figsize=(15,7.5));
fig = ax.figure
fig.suptitle='Magnitude Frequency Distribution'


Of course, you can always plot the data using matplotlib functions!

In [ ]:
fig = plt.figure(figsize=(15,7))
plt.plot(sprops['magnitude'], sprops['shakemap-maxmmi'], 'b.')
tstring = plt.title('Mag vs MMI')
xlabel = plt.xlabel('Magnitude')
ylabel = plt.ylabel('MMI')


## Working with String Data <a name="working-with-string-data"></a>

Let's load in a new dataframe to explore what can be done with strings in pandas columns. This table contains earthquake "pick" information - that is, the time at which the earthquake wave ('P', 'S', or more exotic flavors like 'PKPdf') was detected at the station. 

In this case the arrival time information is split up into multiple **string** fields, instead of being in one datetime field as we might expect. Additionally, each date or time column is preceded by a character denoting what time unit is is ("Y" for "Year", etc.). Let's try to merge these columns into one 'Arrival Time' column which has a datetime type.

In [ ]:
phases = pd.read_excel('us60007fd8_phases_broken.xlsx')
phases.head()

A Series object (remember that extracted rows and columns are Series) has an "accessor" called "str" that allows you to perform string operations on the fields in that Series. Let's use that to fix the date/time columns to be integers.

In [ ]:
phases['year'] = phases['year'].str.replace('Y','')
phases['month'] = phases['month'].str.replace('m','')
phases['day'] = phases['day'].str.replace('D','')
phases['hour'] = phases['hour'].str.replace('H','')
phases['minute'] = phases['minute'].str.replace('M','')
phases['second'] = phases['second'].str.replace('S','')
phases.head()

In [ ]:
timecols = ['year', 'month', 'day', 'hour', 'minute', 'second']
phases['Arrival Time'] = pd.to_datetime(phases[timecols])
phases.head()

Now let's get rid of our unnecessary date/time fields

In [ ]:
phases = phases.drop(labels=timecols, axis='columns')
phases.head()

## Applying Functions to Data <a name="applying-functions-to-data"></a>

Most of the time, you should be able to perform vector operations on your pandas DataFrame. Sometimes, however, you may not be able to do this. Here we'll show the use of the DataFrame `apply()` method to all rows of our data. First, let's write a little function to calculate the average rate the wave traveled.   

In [ ]:
def calc_rate(row):
    # hard coding event time b/c it's not in our data set
    # 2020-01-25 03:03:34.276
    event_time = datetime(2020,1,25,3,3,34)
    dt = row['Arrival Time'] - event_time
    rate = row['Distance']/dt.total_seconds()
    return rate


In [ ]:
phases['Rate (deg/sec)'] = phases.apply(calc_rate, axis='columns')
phases.head()

## Resources <a name="resources"></a>

 - pandas Documentation: https://pandas.pydata.org/
 - pandas Read CSV: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
 - Excellent (4 hour) pandas tutorial from Scipy 2021: https://youtu.be/9dz1fmBUF8U?t=66